In [33]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf

In [34]:
#Data
housing = pd.read_csv("housing.csv")
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [35]:
#Drop the missing values
housing.dropna(axis=0, inplace=True)

# Let's also reset the index
housing.reset_index(inplace=True, drop=True)


housing.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000
mean,-119.570689,35.633221,28.633094,2636.504233,537.870553,1424.946949,499.433465,3.871162,206864.413155
std,2.003578,2.136348,12.591805,2185.269567,421.385070,1133.208490,382.299226,1.899291,115435.667099
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1450.000000,296.000000,787.000000,280.000000,2.563700,119500.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.536500,179700.000000
75%,-118.010000,37.720000,37.000000,3143.000000,647.000000,1722.000000,604.000000,4.744000,264700.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [36]:
#DATA CLEANING
#Set the training and test data sets
housing_num = housing.drop("ocean_proximity", axis=1) # drop labels 

#Select the label
housing_target = housing[["ocean_proximity"]]


In [37]:
#STANDERDISING THE DATA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

housing_num_std = scaler.fit_transform(housing_num)

In [38]:
housing_num_std

array([[-1.32731375,  1.05171726,  0.98216331, ..., -0.97683327,
         2.34516291,  2.12881864],
       [-1.32232256,  1.04235526, -0.60621017, ...,  1.67037262,
         2.33263161,  1.31362603],
       [-1.33230494,  1.03767426,  1.85576873, ..., -0.84342665,
         1.78293943,  1.25818254],
       ...,
       [-0.82320322,  1.77727236, -0.92388486, ..., -0.17377773,
        -1.14317103, -0.99247676],
       [-0.87311515,  1.77727236, -0.84446619, ..., -0.39350628,
        -1.05513604, -1.05831591],
       [-0.83318561,  1.74918635, -1.00330353, ...,  0.07995643,
        -0.78060586, -1.01759959]])

In [39]:
housing_num_std.shape

(20433, 9)

In [40]:
#USING ENCOADING TO CONVERT THE DATA TO INTEGERS

In [41]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()

housing_labels_ord = ordinal_encoder.fit_transform(housing_target)

housing_labels_ord[:10]

array([[3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.]])

In [42]:
#Converting to integer

housing_labels = housing_labels_ord.astype(int)
housing_labels.dtype

dtype('int64')

In [43]:
housing_labels.shape

(20433, 1)

In [44]:
#SPLIT THE DATA
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(housing_num_std, housing_labels, test_size=0.3)

In [45]:
#MULTICLASS CLASSIFICATION USING KERAS

In [46]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

# fix random seed for reproducibility
np.random.seed(42)

In [47]:
#Defining the model: for multi-class

model = Sequential()

model.add(Dense(50, input_dim=9, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(10, activation='relu'))

#final layer: there has to be 5 nodes with softmax (because we have 5 categories)
model.add(Dense(5, activation='softmax'))


In [48]:
# Compile model

#Optimizer:
sgd = keras.optimizers.SGD(lr=0.05)


#You need to use "categorical_crossentropy" for mutli-class
#but since our target is ordinal, we need to use "sparse_..."
#if it is binary classification, then use binary_crossentropy

model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [49]:
# Fit the model

model.fit(train_x, train_y, epochs=100, batch_size=100)


Epoch 1/100
144/144 [==============================] - 1s 1ms/step - loss: 1.1037 - accuracy: 0.5930
Epoch 2/100
144/144 [==============================] - 0s 2ms/step - loss: 0.7317 - accuracy: 0.7375
Epoch 3/100
144/144 [==============================] - 0s 1ms/step - loss: 0.6167 - accuracy: 0.7708
Epoch 4/100
144/144 [==============================] - 0s 1ms/step - loss: 0.5491 - accuracy: 0.7856
Epoch 5/100
144/144 [==============================] - 0s 1ms/step - loss: 0.4813 - accuracy: 0.8016
Epoch 6/100
144/144 [==============================] - 0s 1ms/step - loss: 0.4256 - accuracy: 0.8330
Epoch 7/100
144/144 [==============================] - 0s 2ms/step - loss: 0.3854 - accuracy: 0.8435
Epoch 8/100
144/144 [==============================] - 0s 1ms/step - loss: 0.3661 - accuracy: 0.8509
Epoch 9/100
144/144 [==============================] - 0s 1ms/step - loss: 0.3558 - accuracy: 0.8553
Epoch 10/100
144/144 [==============================] - 0s 1ms/step - loss: 0.3837 - accura

In [50]:
# evaluate the model

scores = model.evaluate(test_x, test_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

192/192 [==============================] - 0s 1ms/step - loss: 0.2864 - accuracy: 0.9078

accuracy: 90.78%


In [51]:
model1 = Sequential()

model1.add(Dense(10, input_dim=9, activation='relu'))
model1.add(Dense(25, activation='relu'))
model1.add(Dense(50, activation='relu'))

#final layer: there has to be 5 nodes with softmax (because we have 5 categories)
model1.add(Dense(5, activation='softmax'))

In [52]:


#Optimizer:
sgd = keras.optimizers.SGD(learning_rate=0.05)


#You need to use "categorical_crossentropy" for mutli-class
#but since our target is ordinal, we need to use "sparse_..."
#if it is binary classification, then use binary_crossentropy

model1.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [53]:
model.fit(train_x, train_y, epochs=100, batch_size=100)

Epoch 1/100
144/144 [==============================] - 0s 2ms/step - loss: 0.1528 - accuracy: 0.9369
Epoch 2/100
144/144 [==============================] - 0s 1ms/step - loss: 0.1504 - accuracy: 0.9392
Epoch 3/100
144/144 [==============================] - 0s 1ms/step - loss: 0.1551 - accuracy: 0.9374
Epoch 4/100
144/144 [==============================] - 0s 1ms/step - loss: 0.1557 - accuracy: 0.9356
Epoch 5/100
144/144 [==============================] - 0s 1ms/step - loss: 0.1517 - accuracy: 0.9378
Epoch 6/100
144/144 [==============================] - 0s 2ms/step - loss: 0.1578 - accuracy: 0.9340
Epoch 7/100
144/144 [==============================] - 0s 1ms/step - loss: 0.1515 - accuracy: 0.9383
Epoch 8/100
144/144 [==============================] - 0s 1ms/step - loss: 0.1538 - accuracy: 0.9358
Epoch 9/100
144/144 [==============================] - 0s 1ms/step - loss: 0.1542 - accuracy: 0.9357
Epoch 10/100
144/144 [==============================] - 0s 1ms/step - loss: 0.1492 - accura

In [54]:
# evaluate the model

scores = model.evaluate(test_x, test_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

192/192 [==============================] - 0s 1ms/step - loss: 1.6398 - accuracy: 0.7613

accuracy: 76.13%


In [56]:
#From the above we can say that the First approach has the accuracy of  90%, So first approach is working better in both the models. That is by decreasing the neurons the model is better compared to increasing the neurons in the layers.